In [1]:
import sys, os
import pandas as pd
# import setting
import pathSetting
sys.path.append("../../..")

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import model as ml_model
from Clust.clust.ML.tool import clean as ml_clean

import torch

#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


# 2. Training 

## 2-1. Data selection

In [2]:
# dataRoot = 'ml_data_integration'

# 1 (p2부분 1. Data Selection)
DataMeta = p1.read_json_data(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())

In [3]:
dataList

['a5d2781fb26f1520fcee266873deeaa6_4d7849735730d38f0780953d5d57490b',
 'a00e9931e8fe54b78d610e464ca4a4e8_4d7849735730d38f0780953d5d57490b',
 'NoCleanHS1Train',
 'NoCleanHS1Test',
 'CleanHS1Test',
 'CleanHS1Train',
 'TempHS1NoCleanTrain',
 'TempHS1NoCleanTest',
 'HS1NoCleanTrain1',
 'HS1NoCleanTest1',
 'trainClean_Hs1SwineFarmWithWeatherTime',
 'trainNoClean_Hs1SwineFarmWithWeatherTime',
 'testClean_Hs1SwineFarmWithWeatherTime',
 'testNoClean_Hs1SwineFarmWithWeatherTime',
 'CO2HS2CleanForecasting',
 'CO2HS2NoCleanForecasting',
 'CleanHS2Train',
 'NoCleanHS2Train',
 'CleanHS2Test',
 'NoCleanHS2Test']

In [4]:
# # for regression data
# #2
# # dataX
# dataName_X = dataList[0]
# dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

# # datay
# dataName_y = dataList[1]
# dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]

# #3
# dataFolderName = "data_integrated_result"
# current = os.getcwd()
# dataFolderPath = os.path.join(current, dataFolderName)
# dataX = ml_data.get_saved_integrated_data(dataSaveMode_X, dataName_X, dataFolderPath)
# datay = ml_data.get_saved_integrated_data(dataSaveMode_y, dataName_y, dataFolderPath)
# integration_freq_sec = DataMeta[dataName_X]["integrationInfo"]["integration_freq_sec"]

In [5]:
# for forecast data test
cleanParamList = ['Clean', 'NoClean']
cleanMode = cleanParamList[1]

datasetNameList = ['Hs1SwineFarmWithWeatherTime', 'gunwiStrawberryWithWeatherTime', 'strawberryOpenTime']
datasetName = datasetNameList[0]

dataName_X = 'train' + cleanMode + '_' + datasetName
print(dataName_X)
dataSaveMode_X = DataMeta[dataName_X]['integrationInfo']['DataSaveMode']

dataX = ml_data.get_saved_integrated_data(dataSaveMode_X, dataName_X, pathSetting.dataFolderPath)
integration_freq_sec = DataMeta[dataName_X]['integrationInfo']['integration_freq_sec']

trainNoClean_Hs1SwineFarmWithWeatherTime


## 2-2. Training Data Preparation

In [6]:
# # for regression
# # 2 Training Data Preparation
# # 2-1
# featureListX= list(dataX.columns)
# featureListy= list(datay.columns)

# # 2-2
# cleanTrainDataParam = 'NoClean'#  Classification, Regression과 같이 X, y가 분리된 경우에는 현재 고정해서 사용해야함

# # 2-2-1 cleanTrainDataParam == Clean 일 경우
# NaNProcessingParam ={
#     "feature_cycle":'Day',
#     "feature_cycle_times":1,
#     "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
# }
# # 2-3
# scalerParam='scale'
# scaleMethod='minmax'

# # 2-4
# splitRatio = 0.8

# # 2-5
# scalerRootPath_X = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
# scalerRootPath_y = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
# train_x, val_x, X_scalerFilePath = ml_data.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod)
# train_y, val_y, y_scalerFilePath = ml_data.get_train_val_data(datay, featureListy, scalerRootPath_y, splitRatio, scalerParam, scaleMethod)

In [7]:
# for forecast
# 2-1
featureListX = ['Temperature', 'out_temp','sin_hour']
target_col = 'Temperature'

# 2-2
cleanTrainDataParam = cleanMode

# 2-2-1 cleanMode == Clean 일 때만 활성화
NaNProcessingParam ={
    "feature_cycle":'Day',
    "feature_cycle_times":1,
    "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
}

# 2-3
scalerParam='scale'
scaleMethod ='robust'

# 2-4
splitRatio = 0.8

# 2-5
scalerRootPath_X = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanMode)
train_x, val_x, X_scalerFilePath = ml_data.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod)
train_x, val_x = ml_clean.delete_low_quality_train_val_data(train_x, val_x, cleanMode, NaNProcessingParam)

['Temperature', 'out_temp', 'sin_hour']
Make New scaler File
------ {'feature_cycle': 'Day', 'feature_cycle_times': 1, 'NanInfoForCleanData': {'type': 'num', 'ConsecutiveNanLimit': 3, 'totalNaNLimit': 30000}}


In [8]:
# Transform

# optional? forecasting?
transformParameter = {
    'future_step': 2,
    'past_step': 24,
    'feature_col': featureListX,
    'target_col': target_col,
    'clean_param': cleanTrainDataParam
}

# regression
# from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf

# trainX, trainy = transDFtoNP(train_x, train_y)
# valX, valy = transDFtoNP(val_x, val_y)

# forecast
from Clust.clust.transformation.purpose.machineLearning import LSTMData

LSTMD = LSTMData()
trainX, trainy = LSTMD.transform_Xy_arr(train_x, transformParameter, transformParameter['clean_param'])
valX, valy = LSTMD.transform_Xy_arr(val_x, transformParameter, transformParameter['clean_param'])

1. Original Data Lenagh: 8514
2. Removed Data Length: 0
3. Clean Data Leangth: 8491
1. Original Data Lenagh: 2127
2. Removed Data Length: 0
3. Clean Data Leangth: 2104


## 2-3 Training 

In [9]:
# # 3-1.
# model_list = ["LSTM_rg","CNN_1D_rg","GRU_rg", "LSTM_FCNs_rg"]
# model_method = model_list[2]

# #import main_regression as mr

# n_epochs = 10 # 학습 epoch 횟수, int(default: 1000, 범위: 1 이상)
# batch_size = 16  # batch 크기, int(default: 16, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)

# trainParameter = setting.modelTestconfig[model_method]
# trainParameter['device']  = device
# trainParameter['n_epochs'] = n_epochs
# trainParameter['batch_size'] = batch_size

# trainParameter['input_size'] = input_size
# trainParameter['seq_len'] = seq_len

# modelTags =["aaaaa"]
# trainDataType = "timeseries"
# modelPurpose = "regression"

# # 2
# trainDataInfo = DataMeta[dataName_X]['integrationInfo']

# # 3. 모델을 저장할 파일 패스를 생성한다.

# model_name = 'EnergyLSTMRegression'
# from Clust.clust.transformation.general.dataScaler import encode_hash_style
# trainParameter_encode =  encode_hash_style(str(trainParameter))
# # trainDataPathList = ["CLUST","Electronics", dataName_X, trainParameter_encode]
# trainDataPathList = [model_name, dataName_X, trainParameter_encode]
# modelFilePath = ml_model.get_model_file_path(trainDataPathList, model_method)

In [12]:
input_size, seq_len = trainX.shape[1], trainX.shape[2]
print(input_size, seq_len)

# RNN model parameters
modelParameter = {
    'rnn_type': 'gru',
    'input_size': input_size, 
    'hidden_size': 64,
    'num_layers': 2,
    'output_dim': 1, 
    'dropout': 0.1, 
    'bidirectional': True
}

# CNN_1D model parameters
# modelParameter = {
#     'input_size': input_size,
#     'seq_len': seq_len,
#     'output_channels': 64,
#     'kernel_size': 3,
#     'stride': 1,
#     'padding': 0, 
#     'dropout': 0.1
# }

# LSTM_FCNs model parameters
# modelParameter = {
#     'input_size': input_size,
#     'num_layers': 2,
#     'lstm_dropout': 0.4,
#     'fc_dropout': 0.1
# }

# FC model parameters
# modelParameter = {
#     'input_size': input_size,
#     'dropout': 0.1,
#     'bias': True
# }

trainParameter = {
    'lr': 1e-4,
    'weight_decay': 1e-6, 
    'device': 'cpu', 
    'n_epochs': 10, 
    'batch_size': 16
}

#cleanParameter = 'NoClean'
model_method = 'GRU'

24 3


In [13]:
# 4. Training
from Clust.clust.ML.regression_YK.train import RegressionTrain as RML
rml = RML()
rml.set_param(trainParameter)
rml.set_model(model_method, modelParameter)
rml.set_data(trainX, trainy, valX, valy)
rml.train()
rml.save_best_model("./yk_LSTM_fore_test.pkl")

Start training model
[1/10] Training loss: 0.0447	 Validation loss: 0.0041
[2/10] Training loss: 0.0085	 Validation loss: 0.0030
[3/10] Training loss: 0.0065	 Validation loss: 0.0025
[4/10] Training loss: 0.0048	 Validation loss: 0.0020
[5/10] Training loss: 0.0039	 Validation loss: 0.0018
[6/10] Training loss: 0.0033	 Validation loss: 0.0014
[7/10] Training loss: 0.0030	 Validation loss: 0.0014
[8/10] Training loss: 0.0029	 Validation loss: 0.0011
[9/10] Training loss: 0.0025	 Validation loss: 0.0010
[10/10] Training loss: 0.0025	 Validation loss: 0.0011

Training complete in 0m 53s
